In [1]:
import os
import glob
import pandas as pd
neg = pd.read_table("neg.txt",encoding="utf-8")
pos = pd.read_table("pos.txt",encoding="utf-8")
neg["tag"]=0
pos["tag"]=1
negd = pd.DataFrame(neg["content"].astype(str))
posd = pd.DataFrame(pos["content"].astype(str))
x_n = negd["content"]
p_p = posd["content"]
y_n = neg["tag"]
y_p = pos["tag"]
x = pd.concat([x_n,p_p])
y = pd.concat([y_n,y_p])
sum=0
for i in x:
  sum = sum+len(i)
a = sum/len(x)
print(a)

36.05279511918275


In [2]:
import jieba
jieba.set_dictionary("dict.txt")

def poemcut(s):
    s = " ".join(jieba.cut(s))
    return s
x_cut = x.apply(poemcut)
x_cut

Building prefix dict from /content/dict.txt ...
Loading model from cache /tmp/jieba.ueb620ec8402181953a0299d7957c0d6e.cache
Loading model cost 1.374 seconds.
Prefix dict has been built successfully.


0                      也許 a13 現今 成本 較 低 電池 依然 貴 的 原因 吧 猜 的
1        沒有 多 鏡頭 沒有 fhd 高畫質 螢幕 iphonese 強到 只 剩快 50 萬 的 ...
2                                 就算 玩遊戲 再順 螢幕 太小 還是 不好玩 n
3                                                 使用者 評論 n
4                    反正 沒差 啦 慢慢來 沒關係 的 啦 要 買 的 人 還是 會 願意 等
                               ...                        
38004    真的 很漂亮 n 最 吸引 我 的 是 台灣版 直接 是 金屬 錶帶 我 戴 塑膠 表帶 都...
38005                               當然 可以 啊 只要 有 藍芽 就 可以 用
38006    野貓 說 我 今天 才 拿到 手 就 送修 了 狂 斷線 nn 希望 早點 換 一只 新 的...
38007                             我 的 手錶 全新 回來 了 哈哈哈 真的 好用
38008    s400 其實 很 夠用 了 不過 c3 主 鏡頭 拍照 的 感覺 不是 很 理想 好像 回...
Name: content, Length: 70480, dtype: object

In [0]:
from tensorflow.keras.preprocessing.text import Tokenizer
# 出現太少的詞, 你可以選擇不看, 只留出現次數最高的2000(num_words)
tok = Tokenizer(num_words=2000)
tok.fit_on_texts(x_cut)

In [4]:
# 文字轉成數字
x_seq = tok.texts_to_sequences(x_cut)
pd.DataFrame(x_seq)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,970,971,972,973,974,975,976,977,978,979,980,981,982,983,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999,1000,1001,1002,1003,1004,1005,1006,1007,1008,1009
0,1302.0,1478.0,452.0,495.0,421.0,154.0,1236.0,207.0,1.0,603.0,21.0,1243.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,31.0,50.0,43.0,31.0,25.0,1429.0,200.0,1567.0,370.0,1.0,380.0,375.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,270.0,717.0,25.0,1152.0,22.0,16.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,284.0,16.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,289.0,548.0,119.0,1294.0,1.0,119.0,19.0,30.0,1.0,33.0,22.0,14.0,797.0,88.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70475,9.0,397.0,16.0,97.0,194.0,10.0,1.0,3.0,189.0,3.0,773.0,10.0,1682.0,8.0,14.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
70476,231.0,15.0,40.0,290.0,6.0,785.0,11.0,15.0,27.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
70477,53.0,10.0,788.0,67.0,772.0,673.0,11.0,2.0,172.0,64.0,477.0,82.0,1.0,1011.0,9.0,4.0,13.0,27.0,401.0,1027.0,560.0,636.0,1.0,265.0,53.0,754.0,15.0,27.0,44.0,16.0,231.0,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
70478,10.0,1.0,1270.0,601.0,1011.0,2.0,650.0,9.0,622.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# NLP沒辦法接受不同的字數,因此必須讓所有句子的字數一樣進去處理
from tensorflow.keras.preprocessing.sequence import pad_sequences
x_seq_pad = pad_sequences(x_seq, maxlen=64)
pd.DataFrame(x_seq_pad)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1302,1478,452,495,421,154,1236,207,1,603,21,1243,1
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,31,50,43,31,25,1429,200,1567,370,1,380,375
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,270,717,25,1152,22,16
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,284,16
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,289,548,119,1294,1,119,19,30,1,33,22,14,797,88
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70475,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,9,397,16,97,194,10,1,3,189,3,773,10,1682,8,14,1
70476,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,231,15,40,290,6,785,11,15,27
70477,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,53,10,788,67,772,673,11,2,172,64,477,82,1,1011,9,4,13,27,401,1027,560,636,1,265,53,754,15,27,44,16,231,15
70478,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,10,1,1270,601,1011,2,650,9,622


In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Flatten , Dropout, Dense
layers=[
        #  2001*64 =128064
        Embedding(2001, 64, mask_zero=True, input_length=64),
        Flatten(),
        Dense(64,activation="relu"),
        Dropout(0.25),
        Dense(2,activation="softmax")
]
model = Sequential(layers)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 64, 64)            128064    
_________________________________________________________________
flatten (Flatten)            (None, 4096)              0         
_________________________________________________________________
dense (Dense)                (None, 64)                262208    
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 130       
Total params: 390,402
Trainable params: 390,402
Non-trainable params: 0
_________________________________________________________________


In [0]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy
model.compile(loss=SparseCategoricalCrossentropy(),
              optimizer="adam",
              metrics=["accuracy"]
)

In [8]:
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint
callbacks=[
           EarlyStopping(patience=3,restore_best_weights=True),
]

model.fit(x_seq_pad,
          np.array(y),
          batch_size = 200,
          epochs=4,
          validation_split=0.1,
          verbose=2,
          callbacks=callbacks)

Epoch 1/4
318/318 - 3s - loss: 0.2187 - accuracy: 0.9064 - val_loss: 0.2460 - val_accuracy: 0.9103
Epoch 2/4
318/318 - 3s - loss: 0.0872 - accuracy: 0.9690 - val_loss: 0.2781 - val_accuracy: 0.9032
Epoch 3/4
318/318 - 3s - loss: 0.0595 - accuracy: 0.9797 - val_loss: 0.3906 - val_accuracy: 0.8753
Epoch 4/4
318/318 - 2s - loss: 0.0360 - accuracy: 0.9886 - val_loss: 0.3293 - val_accuracy: 0.9106


In [11]:
model.evaluate(x_seq_pad, np.array(y))

2203/2203 [==============================] - 7s 3ms/step - loss: 0.0991 - accuracy: 0.9657


[0.09909209609031677, 0.9657207727432251]